In [1]:
# Load Packages
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px

import warnings
import os, sys
from typing import List
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import keras

rootpath = ".."
sys.path.insert(0, f"{os.getcwd()}/{rootpath}/base_models")
warnings.filterwarnings("ignore")

import model_prep


step_back = 6  # window size = 6*5 = 30 mins
season_map = {
    "spring": [3, 4, 5],
    "summer": [6, 7, 8],
    "fall": [9, 10, 11],
    "winter": [12, 1, 2],
}

2023-11-25 02:43:20.919321: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from_building_name = "ESB"
from_tower_number = 2
to_building_name = "ESB"
to_tower_number = 1
to_features = ['FlowEvap', 'PerHumidity', 'TempAmbient', 'TempCondIn',
    'TempCondOut', 'TempEvapIn', 'TempEvapOut', 'TempWetBulb',
    'PerFreqConP', 'Tonnage','DayOfWeek', 'HourOfDay', 'PerFreqFan']
to_target = 'EnergyConsumption'
to_season = "fall"
from_season = "fall"
finetuning_percentage = 0.8
source_epochs=100
finetune_epochs = 100
display_results = True
use_delta = True
shuffle_seed = 42

In [3]:
"""
1. Convert data into a model-compatible shape
"""

lstm_df, first_val = model_prep.create_preprocessed_lstm_df(
    building_name=from_building_name,
    tower_number=from_tower_number,
    features=to_features,
    target=to_target,
    season=from_season,
    use_delta=use_delta,
)

"""
2. Split data into training and testing sets
"""

X = lstm_df.drop(f"{to_target}(t)", axis=1)  # drop target column
y = lstm_df[f"{to_target}(t)"]  # only have target column

# split into input and outputs
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=shuffle_seed
)

# scale feature data
scaler = MinMaxScaler().fit(X_train)
X_train[X_train.columns] = scaler.transform(X_train)
X_test[X_test.columns] = scaler.transform(X_test)

"""
3. Get timestepped data as a 3D vector
"""
vec_X_train = model_prep.df_to_3d(
    lstm_dtframe=X_train, num_columns=len(to_features) + 1, step_back=step_back
)
vec_X_test = model_prep.df_to_3d(
    lstm_dtframe=X_test, num_columns=len(to_features) + 1, step_back=step_back
)

vec_y_train = y_train.values
vec_y_test = y_test.values

"""
4. Create and Train model
"""
dropout_rate = 0.0
weight_constraint = keras.constraints.MaxNorm(4.0)
batch_size = 32
lstmcells = 64
activation = "relu"
optimizer = "Adam"

base_model = keras.models.Sequential()
base_model.add(
    keras.layers.LSTM(
        lstmcells,
        input_shape=(vec_X_train.shape[1], vec_X_train.shape[2]),
        kernel_constraint=weight_constraint,
        recurrent_dropout=dropout_rate,
        activation=activation,
    )
)
base_model.add(keras.layers.Dense(1))
base_model.compile(loss="mse", optimizer=optimizer)

history = base_model.fit(
    vec_X_train,
    vec_y_train,
    epochs=source_epochs,
    batch_size=batch_size,
    validation_data=(vec_X_test, vec_y_test),
    verbose=0,
    shuffle=False,
)
# make predictions
yhat = base_model.predict(vec_X_test)

"""
5. Display results
"""
results_df = pd.DataFrame(
    {
        "actual": vec_y_test.reshape((vec_y_test.shape[0])),
        "predicted": yhat.reshape((yhat.shape[0])),
    },
    index=y_test.index,
)

if use_delta:
    results_df["actual"] = results_df["actual"] + first_val
    results_df["predicted"] = results_df["predicted"] + first_val

# SAVE ERROR AND DATA AVAILABILITY INFORMATION

# calculate, display and save error results
base_mae = mean_absolute_error(results_df["actual"], results_df["predicted"])
base_rmse = np.sqrt(mean_squared_error(results_df["actual"], results_df["predicted"]))
base_mae_sd = np.std(np.abs(results_df["actual"] - results_df["predicted"]))

number of times the hvac is on (energy consumption is > zero) False    14066
True     11899
Name: EnergyConsumption, dtype: int64


2023-11-25 02:43:24.744949: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


75/75 [==============================] - 0s 1ms/step


In [4]:
# fix inputs
if from_season == None and to_season != None:
    from_season = to_season

"""
1. Load data and do LSTM preprocessing
"""

lstm_to_df, to_first_val = model_prep.create_preprocessed_lstm_df(
    building_name=to_building_name,
    tower_number=to_tower_number,
    features=to_features,
    target=to_target,
    season=to_season,
    use_delta=use_delta,
)
if not to_season:
    to_season = from_season = "allyear"
training_time = 0

"""
2. Convert tower data into a model-compatible shape i.e. get timestepped data as a 3D vector
"""

X = lstm_to_df.drop(f"{to_target}(t)", axis=1)  # drop target column
y = lstm_to_df[f"{to_target}(t)"]  # only have target column


number of times the hvac is on (energy consumption is > zero) True     14089
False    11885
Name: EnergyConsumption, dtype: int64


In [5]:
# split train and test set
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=(1 - finetuning_percentage),
    shuffle=False,
    random_state=shuffle_seed,
)

# scale feature data
scaler = MinMaxScaler()
scaler = scaler.fit(X_train)
X_train[X_train.columns] = scaler.transform(X_train)
X_test[X_test.columns] = scaler.transform(X_test)

# create 3d vector form of data
vec_X_train = model_prep.df_to_3d(
    lstm_dtframe=X_train, num_columns=len(to_features) + 1, step_back=step_back
)
vec_X_test = model_prep.df_to_3d(
    lstm_dtframe=X_test, num_columns=len(to_features) + 1, step_back=step_back
)

vec_y_train = y_train.values
vec_y_test = y_test.values

In [6]:
def finetune(
    model: keras.engine.sequential.Sequential,
    training_feature_vec: np.ndarray,
    training_target_vec: np.ndarray,
    epochs: int = 10,
    ):
    model.trainable = True

    model.compile(
        optimizer=keras.optimizers.Adam(1e-5),  # Very low learning rate
        loss="mse",
        metrics=[keras.metrics.BinaryAccuracy()],
    )

    model.fit(
        training_feature_vec,
        training_target_vec,
        epochs=epochs,
        verbose=0,
        shuffle=False
    )

    return model

In [7]:
# load and finetune model
print(f"Finetuning for ft={finetuning_percentage} seed={shuffle_seed}")

start_time = time.time()
model = finetune(
    model=base_model,
    training_feature_vec=vec_X_train,
    training_target_vec=vec_y_train,
    epochs=finetune_epochs,
)
end_time = time.time()
training_time = end_time - start_time

Finetuning for ft=0.8 seed=42


In [8]:
"""
3. Load model, finetune and predict
"""

yhat = model.predict(vec_X_test)

"""
4. Display results
"""

# save results
results_df = pd.DataFrame(
    {
        "actual": vec_y_test.reshape((vec_y_test.shape[0])),
        "predicted": yhat.reshape((yhat.shape[0])),
    },
    index=y_test.index,
)

if use_delta:
    results_df["actual"] = results_df["actual"] + to_first_val
    results_df["predicted"] = results_df["predicted"] + to_first_val

rmse = np.sqrt(mean_squared_error(results_df["actual"], results_df["predicted"]))
mabs_error = mean_absolute_error(results_df["actual"], results_df["predicted"])

# display results
def display_transfer_results():
    # Create a new DataFrame with the desired 5-minute interval index, and merge the new DataFrame with the original DataFrame
    display_df = pd.DataFrame(
        index=pd.date_range(
            start=results_df.index.min(), end=results_df.index.max(), freq="5min"
        )
    ).merge(results_df, how="left", left_index=True, right_index=True)

    print("RMSE: %.3f" % rmse)

    fig = px.line(display_df, x=display_df.index, y=["actual", "predicted"])
    fig.update_layout(
        title=f"{from_building_name} Tower {from_tower_number} {from_season} model used on {to_building_name} Tower {to_tower_number} {to_season} ({finetuning_percentage*100}% fine-tuning) LSTM Model Results",
        xaxis_title="time",
        yaxis_title=to_target,
    )
    return fig

if display_results:
    fig = display_transfer_results()
else:
    fig = None

88/88 [==============================] - 0s 1ms/step
RMSE: 8.036


In [9]:
fig

In [10]:
results_df[results_df["predicted"]<0]

,actual,predicted
time,,


In [11]:
mabs_error, base_mae

(6.3162243386664905, 7.269507310489855)